Scraping data from url pages
in this file we will collect data from fake and real sources and save the dataframes as csv files


In [188]:
import os
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from urllib.request import urlopen

In [189]:
# loading dataset: we have real and fake one
df_fake=pd.read_csv('gossipcop_fake_2.csv')
print(df_fake.shape)


(5323, 4)


In [190]:
df_real=pd.read_csv('gossipcop_real_2.csv')
print(df_real.shape)

(1026, 4)


In [191]:
# we are going to keep 300 urls from each type of sources:
df_real.head(3)

,id,news_url,title,tweet_ids
0,gossipcop-882573,https://www.brides.com/story/teen-mom-jenelle-...,Teen Mom Star Jenelle Evans' Wedding Dress Is ...,912371411146149888\t912371528343408641\t912372...
1,gossipcop-875924,https://www.dailymail.co.uk/tvshowbiz/article-...,Kylie Jenner refusing to discuss Tyga on Life ...,901989917546426369\t901989992074969089\t901990...
2,gossipcop-894416,https://en.wikipedia.org/wiki/Quinn_Perkins,Quinn Perkins,931263637246881792\t931265332022579201\t931265...


In [121]:
#is_NaN = df_real_fl.isnull()
#row_has_NaN = is_NaN.any(axis=1)
#rows_with_NaN = df_real_fl[row_has_NaN]
#rows_with_NaN

In [ ]:
#we will take around 100 articles per type

In [239]:
# CREATING THe real and fake dataframes to store the new data
df_real_fl = pd.DataFrame(df_real.loc[:115,["news_url","title"]])
df_fake_fl = pd.DataFrame(df_fake.loc[:118,["news_url","title"]])
df_real_fl.dropna( inplace=True )
df_fake_fl.dropna( inplace=True )

In [240]:

df_real_fl.reset_index(drop=True, inplace=True)
df_fake_fl.reset_index(drop=True, inplace=True)
print(df_fake_fl.shape)
print(df_real_fl.shape)

(114, 2)
(116, 2)


In [258]:

def extract_text_body(doc):
    # url of the website: doc

    try:
        res = requests.get(doc)
    except requests.ConnectionError:
        print("Can't connect to the site, sorry")
        return float("NaN")
    else:
        soup = BeautifulSoup(res.content, "html.parser")

        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out


        #print(res.status_code)
        #print(soup)
        if res.status_code == 200:
            # get text from body
            text = soup.body.get_text()
            # break into lines and remove leading and trailing space on each
            lines = (line.strip() for line in text.splitlines())
            # break multi-headlines into a line each
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            # drop blank lines
            text = ' '.join(str(chunk) for chunk in chunks if chunk)

        else:
            text = float("NaN")


        return text

In [259]:
for i in range(df_fake_fl.shape[0]):
#    print(i)
    df_real_fl.loc[i,"text"]=extract_text_body(df_real_fl.loc[i,"news_url"])
    df_fake_fl.loc[i,"text"]=extract_text_body("http://"+df_fake_fl.loc[i,"news_url"])



Can't connect to the site, sorry
Can't connect to the site, sorry
Can't connect to the site, sorry
Can't connect to the site, sorry
Can't connect to the site, sorry
Can't connect to the site, sorry


In [260]:
for i in range(df_fake_fl.shape[0],df_real_fl.shape[0]):
    #print(i)
    df_real_fl.loc[i,"text"]=extract_text_body(df_real_fl.loc[i,"news_url"])


In [261]:

# getting the source/domain from url as source:
df_real_fl.loc[:,"source"]=[urlparse(url).netloc for url in df_real_fl.loc[:,"news_url"]]
df_fake_fl.loc[:,"source"]=[urlparse("http://"+url).netloc for url in df_fake_fl.loc[:,"news_url"]]

In [262]:
# attributing the type of data to each df
df_real_fl["type"]="real"
df_fake_fl["type"]="fake"

In [268]:
#[len(text) for text in df_fake_fl.loc[:,"text"] if type(text)==str]

In [269]:
df_fake_fl.head(5)

,news_url,title,text,source,type
0,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,Home U.K. News Sports U.S. Showbiz Australia F...,www.dailymail.co.uk,fake
1,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,NaN,hollywoodlife.com,fake
2,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,× Plus Icon Click to expand the Mega Menu Plus...,variety.com,fake
3,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,Home U.K. News Sports U.S. Showbiz Australia F...,www.dailymail.co.uk,fake
4,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,Plus Icon Click to expand the Mega Menu Plus I...,variety.com,fake


In [ ]:
#df_real_fl.dropna( inplace = True)
#df_fake_fl.dropna(inplace = True)

In [270]:
# saving oir df into CSV files for future analysis:
df_real_fl.to_csv('gossip_full_real.csv', index=False )
df_fake_fl.to_csv('gossip_full_fake.csv',index=False)

In [271]:
df_fake_fl["text"].shape

(114,)